# 캐글 데이터 다운로드 받는 법
- 캐글에서 받은 API key 필요

In [3]:
from google.colab import drive

drive.mount('/gdrive')


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [4]:
import os, json

with open('kaggle.json') as f:
    kaggle = json.load(f)
os.environ['KAGGLE_USERNAME']= kaggle['username']
os.environ['KAGGLE_KEY']=kaggle['key']


In [5]:
!kaggle datasets download -d shaunthesheep/microsoft-catsvsdogs-dataset


 99% 777M/788M [00:03<00:00, 215MB/s]
100% 788M/788M [00:03<00:00, 241MB/s]


In [10]:
!unzip microsoft-catsvsdogs-dataset.zip > /dev/null

replace MSR-LA - 3467.docx? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [14]:
cat_dir= 'PetImages/Cat'
os.unlink(f'{cat_dir}/Thumb.db')
cat_list = os.listdir(cat_dir)
len(cat_list)

FileNotFoundError: ignored

In [15]:
dog_dir= 'PetImages/Dog'
os.unlink(f'{dog_dir}/Thumb.db')
dog_list = os.listdir(dog_dir)
len(dog_list)

FileNotFoundError: ignored

미지 파일 에러 찾기

In [8]:
from PIL import Image
from glob import glob

In [12]:
for fname in cat_list+dog_list:
    try:
        img=Image.open(f'PetImages/Cat/{fname}')
    except:
        print(fname)        

NameError: ignored

In [ ]:
!cp PetImages/Cat/10000.jpg PetImages/Cat/666.jpg
!cp PetImages/Dog/10000.jpg PetImages/Dog/11702.jpg

0~7999.jpg=train
8000~9999 = valid
10000~12499= test